# Inverse Propensity Score

One of the most fundamental features of _RecoGym_ is the ability to collect logs of applying a specific agent in the _RecoGym_ environment. That opens a plethora of opportunities to examine machine learning based on _Counterfactual Risk Minimization_ and compare the result with machine learning based on _Empirical Risk Minimization_.

In _[Likelihood Agents](./Likelihood%20Agents.ipynb)_ notebook, some _Agents_ those use different feature sets for _Liklihood_ model have already been analysed. In this notebook, we consider _Inverse Propensity Score_ that is a part of _Counterfactual Risk Minimization_ approach.

# The Data

In [1]:
import gym, reco_gym
from copy import deepcopy
from reco_gym import env_1_args
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [8, 4]

ABTestNumberOfUsers = 10000
RandomSeed = 42

NumberOfProducts = 10

env_1_args['random_seed'] = RandomSeed
env_1_args['num_products'] = NumberOfProducts
env_1_args['number_of_flips'] = 1
env_1_args['num_epochs'] = 100


env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)

In [2]:
data = deepcopy(env).generate_logs(ABTestNumberOfUsers)

In [3]:
print("Data:\n", data[:20])
print("Data Shape:\n", data.shape)

Data:
      t  u        z    v    a    c   ps
0    0  0  organic  0.0  NaN  NaN  NaN
1    1  0   bandit  NaN  3.0  0.0  0.1
2    2  0   bandit  NaN  4.0  0.0  0.1
3    3  0   bandit  NaN  5.0  0.0  0.1
4    0  1  organic  1.0  NaN  NaN  NaN
5    1  1   bandit  NaN  2.0  0.0  0.1
6    2  1   bandit  NaN  8.0  0.0  0.1
7    3  1   bandit  NaN  4.0  0.0  0.1
8    4  1  organic  4.0  NaN  NaN  NaN
9    5  1  organic  4.0  NaN  NaN  NaN
10   6  1  organic  4.0  NaN  NaN  NaN
11   7  1  organic  4.0  NaN  NaN  NaN
12   8  1  organic  4.0  NaN  NaN  NaN
13   9  1  organic  6.0  NaN  NaN  NaN
14  10  1  organic  4.0  NaN  NaN  NaN
15  11  1  organic  4.0  NaN  NaN  NaN
16  12  1  organic  4.0  NaN  NaN  NaN
17  13  1  organic  6.0  NaN  NaN  NaN
18  14  1  organic  4.0  NaN  NaN  NaN
19  15  1  organic  1.0  NaN  NaN  NaN
Data Shape:
 (1005241, 7)


Details regarding the content of the data you shall find in _[Likelihood Agents](./Likelihood%20Agents.ipynb)_ notebook.

Please, note that there is a column _**`ps`**_ that is a _Probability Score_ of selecting a particular _`Action`_ by an _`Agent`_. Since the default _`Agent`_ selects an _`Action`_ _**randomly**_ and as there are _`10`_ different _`Actions`_ in overall, the _Probability Score_ is always $\frac{1}{10}$ = _`0.1`_ for all _Bandit_ _`Events`_.

# Multi-Class Classification Models

All models analised below use _Multi-Class Classification_ where for a provided set of features, the models predict an _`Action`_.

That approach differs to what was used in _Logistic Regression_ for _Emirical Risk Minimization_ where an _`Action`_ was a part of input features.

In [4]:
import math
import numpy as np

def build_train_data(
    data,
    weight_history_function = None
):
    """
    Build Train Data

        Parameters:
            data: offline experiment logs
                the data contains both Organic and Bandit Events
            weight_history_function(function): weight functions that assigns an appropriate weight
                for View in Organic Event for a certain Product

        Returns:
            :(features, outs)
    """
    num_products = int(data.v.max() + 1)
    number_of_users = int(data.u.max()) + 1

    features = []
    actions = []
    pss = []
    clicks = []

    for user_id in range(number_of_users):
        views = np.zeros((0, num_products))
        for _, user_datum in data[data['u'] == user_id].iterrows():
            if user_datum['z'] == 'organic':
                assert (math.isnan(user_datum['a']))
                assert (math.isnan(user_datum['c']))
                assert (not math.isnan(user_datum['v']))

                view = int(user_datum['v'])

                tmp_view = np.zeros(num_products)
                tmp_view[view] = 1

                # Append the latest view at the beginning of all views.
                views = np.append(tmp_view[np.newaxis, :], views, axis = 0)
            else:
                assert (user_datum['z'] == 'bandit')
                assert (not math.isnan(user_datum['a']))
                assert (not math.isnan(user_datum['c']))
                assert (math.isnan(user_datum['v']))

                action = int(user_datum['a'])
                click = int(user_datum['c'])
                ps = int(user_datum['ps'])

                if weight_history_function is None:
                    train_views = views
                else:
                    history = np.array(range(views.shape[0])).reshape(views.shape[0], 1)
                    weights = weight_history_function(history)
                    train_views = views * weights

                feature = np.sum(train_views, axis = 0)

                features.append(feature)
                actions.append(action)
                clicks.append(click)
                pss.append(ps)

    return np.array(features), np.array(actions), np.array(clicks), np.array(pss)

## Logistic Regression with Wieghted Training Data

The model is based on _Logistic Regression_ of multiple classes (different _`Actions`_) where during training each feature set will be weighted. The weight for $i^th$ feature set is actually _Inverse Propensity Score_ that is calculated as follows:



$$ w_i \equiv ips_i = \frac{\delta_i}{p_i} $$

* $
\delta_i =
\begin{cases}
    1,& \text{if there is a click}\\
    0,& \text{otherwise}
\end{cases}
$
* $p_i$ is a _Probability Score_ that is for the case of _Random_ _`Agent`_ is _**always**_ _`0.1`_.

In [ ]:
train_features01, train_actions01, train_clicks01, train_pss01 = build_train_data(data)

random_ps = 1.0 / NumberOfProducts
train_weights01 = train_clicks01 / random_ps

from sklearn.linear_model import LogisticRegression

logreg_weighted01 = LogisticRegression(
    solver = 'lbfgs',
    max_iter = 5000,
    multi_class='multinomial',
    random_state = RandomSeed
)

lr_weighted01 = logreg_weighted01.fit(train_features01, train_actions01, train_weights01)

In [ ]:
lr_weighted01.coef_

Let's check how the model works if in _`Views`_ thare are _`10`_s for each _`Product`_.

In [ ]:
test_views01 = np.zeros((NumberOfProducts, NumberOfProducts))
for product_id in range(NumberOfProducts):
    test_views01[product_id, product_id] = 10
print("Predicted Action: ", lr_weighted01.predict(test_views01))
print("Predicted Probability of Actions: ", lr_weighted01.predict_proba(test_views01))

As you can see, only _`Products`_ _`4`_, _`5`_, and _`9`_ correlate with amount of appropriate _`Views`_.

Let's build a graph to reveal the correlation of matched products with the appropriate amount of _`Views`_.

In [ ]:
all_product_ids = np.array(range(NumberOfProducts))
inc_matrix = np.eye(NumberOfProducts)
views = np.zeros((NumberOfProducts, NumberOfProducts))
counts = []
matcheses = []
for count in range(200):
    views += inc_matrix
    predictions = lr_weighted01.predict(views)
    matches = np.sum(predictions == all_product_ids)
    matcheses.append(matches)
    counts.append(count)

_, ax = plt.subplots()
ax.plot(counts, matcheses)
plt.show()

Now, we will build a more complex model with product crossfitting.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

logreg_weighted02 = LogisticRegression(
    solver = 'lbfgs',
    max_iter = 10000,
    multi_class='multinomial',
    random_state = RandomSeed
)
poly2 = PolynomialFeatures(2)
lr_weighted02 = logreg_weighted02.fit(poly2.fit_transform(train_features01), train_actions01, train_weights01)

In [ ]:
views = np.zeros((NumberOfProducts, NumberOfProducts))
counts = []
matcheses = []
for count in range(200):
    views += inc_matrix
    predictions = lr_weighted02.predict(poly2.fit_transform(views))
    matches = np.sum(predictions == all_product_ids)
    matcheses.append(matches)
    counts.append(count)

_, ax = plt.subplots()
ax.plot(counts, matcheses)
plt.show()

At this moment, it is not clear yet how the model behavies and what kind of performance in _Click-Through Rate_ shall we expect. To reveal that, we bench the _`Agent`_ that uses the model.

## Hypothesis with Inversed Propensity Score

So far, _[scikit-learn](https://scikit-learn.org)_ was used to create models. This time, we are going to build a more complicated model based on a _Neural Network_. The model implements a new hypothesis $h_c(A=a|V)$ that for given _`Views`_ finds _the probability of the click_ for an _`Action`_ $a$. The value of that function is _Probability Score_ that is stored by _RecoGym_ in offline logs.

At this time, we will use _[PyTorch](https://pytorch.org)_ to train models.

$$
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\DeclareMathOperator{\Var}{Var}
\hat{R}^{M}(h_c) = \frac{1}{n} \sum_{i}^{N}\delta_{i} \min \left \{ M, \frac{h_c(A = a_i|V_i)}{p_i} \right \}
$$

$$
u_i = \delta_i \min \left \{ M, \frac{h_c(A = a_i|V_i)}{p_i} \right \}, \bar{u} = \frac{1}{N} \sum_{i}^{N} u_i
$$

$$
\Var_{h_c}(u) = \frac{1}{N - 1} \sum_{i}^{N} \left( u_i - \bar{u} \right)^2
$$

$$
\hat{h}_c^{CRM} = \argmin_{h \in \mathcal{H}} \left \{ \hat{R}^{M}(h_c) + \lambda \sqrt{\frac{\Var_{h_c}(u)}{N}} \right \} 
$$

In the new model, $\hat{h}_c^{CRM}$ is to be minimised, where:
* $\hat{h}_c^{CRM}$ is a new hypothesis that based on _Counterfactual Risk Minimization_
* $N$ is the number of training samples
* $p_i$ is a _Propensity Score_ or a probability of applying the default hypothesis $h_0$; in our case, it is always $\frac{1}{10}=0.1$
* $a_i$ is an _`Action`_ provided in the ith training sample
* $V_i$ _`Views`_ at ith training sample

In the new model, we also use a regularisation defined as: $\lambda \sqrt{\frac{\Var_{h_c}(u)}{N}}$.

The new hypothesis is implemented in a neural network.
The network has the following topology:
* _`2`_ sigmoid layers
* _`1`_ _softmax_ layer

More details about the structure of the NN, you shall find in _`NeuralNet`_ class.

The model used here was motivated by the article _[arXiv:1502.02362](https://arxiv.org/pdf/1502.02362.pdf)_.

## Comparing Agent Performance

Let's compare the performance of models with _IPS & Multi-Class Logistic Regression_ vs _IPS & Hypothesis implemented in Neural Network_.

In [ ]:
import gym, reco_gym
from copy import deepcopy

import pandas as pd
from reco_gym import env_1_args, Configuration
from agents import LogregPolyAgent, logreg_poly_args
from agents import LogregMulticlassIpsAgent, logreg_multiclass_ips_args
from agents import NnIpsAgent, nn_ips_args

NumberOfflineUsers = 10000
NumberTestUsers = 1000

test_env = {
    **env_1_args,
    **{
        'random_seed': RandomSeed,
    }
}

env = gym.make('reco-gym-v1')
env.init_gym(test_env)


def agent_performance(
    agent,
    user_per_step = 1000,
    user_steps = 10,
    test_users = NumberTestUsers
):
    steps = []
    data = {
        '0.025': [],
        'mean': [],
        '0.975': [],
    }
    for step in range(user_steps):
        number_of_offline_users = (step + 1) * user_per_step
        stats = reco_gym.test_agent(
            deepcopy(env),
            agent,
            number_of_offline_users,
            test_users
        )
        steps.append(number_of_offline_users)
        data['mean'].append(stats[0])
        data['0.025'].append(stats[1])
        data['0.975'].append(stats[2])
    
    figs, axs = plt.subplots(
        1,
        1,
        figsize = (16, 8)
    )
    axs.fill_between(
        range(len(data['mean'])),
        data['0.975'],
        data['0.025'],
        alpha=.05
    )
    axs.plot(steps, data['mean'])
    axs.set_xlabel('Samples #')
    axs.set_ylabel('CTR')
    axs.legend(['$\mu$'])
    plt.show()
    
    return pd.DataFrame().from_dict(data)

#### Model: _Logistic Regression_

In [ ]:
stat00 = agent_performance(
    LogregPolyAgent(Configuration({
        **logreg_poly_args,
        **test_env,
        'with_ips': False,
    }))
)

In [ ]:
print("CTR of Logistic Regression:\n", stat00)

#### Model: _Logistic Regression with Inverse Propensity Score_

In [ ]:
stat01 = agent_performance(
    LogregPolyAgent(Configuration({
        **logreg_poly_args,
        **test_env,
        'with_ips': True,
    }))
)

In [ ]:
print("CTR of Logistic Regression & IPS:\n", stat01)

#### Model: _Multiclass Logistic Regression with Inverse Propensity Score_

In [ ]:
stat02 = agent_performance(
    LogregMulticlassIpsAgent(Configuration({
        **logreg_multiclass_ips_args,
        **test_env,
    }))
)

In [ ]:
print("CTR of Multi-Class Logistic Regression & IPS:\n", stat02)

#### Model: _Hypothesis by Neural Network with Inverse Propensity Score_

In [ ]:
stat03 = agent_performance(
    NnIpsAgent(Configuration({
        **nn_ips_args,
        **test_env,
    }))
)

In [ ]:
print("CTR of Hypothesis by Neural Network & IPS:\n", stat03)

## Conclusion

$$  $$